# Testing Young Child Bonus Feature

This notebook tests whether the young child bonus is correctly applied to:
1. Individual household calculations
2. Statewide impact calculations

In [2]:
import sys
from pathlib import Path
import numpy as np
from policyengine_us import Simulation

# Add parent directory to path
sys.path.insert(0, str(Path.cwd()))

from ri_ctc_calc.calculations.household import build_household_situation
from ri_ctc_calc.calculations.reforms import create_custom_reform
from ri_ctc_calc.calculations.microsimulation import calculate_aggregate_impact

# Check environment
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if young_child_boost parameters exist
test_sim = Simulation(situation={
    "people": {"person": {"age": {"2026": 30}}},
    "households": {"household": {"members": ["person"]}},
    "tax_units": {"tax_unit": {"members": ["person"]}},
})

try:
    boost_amount = test_sim.tax_benefit_system.parameters.gov.contrib.states.ri.ctc.young_child_boost.amount('2026-01-01')
    print(f"\nYoung child boost parameters exist!")
    print(f"Default boost amount: ${boost_amount}")
except Exception as e:
    print(f"\nERROR: Young child boost parameters NOT found!")
    print(f"Error: {e}")
    print("\nYour Jupyter kernel may be using a different Python environment.")
    print("Please restart the kernel and ensure it's using the same Python as your terminal.")

Python executable: c:\Users\dtsax\AppData\Local\Programs\Python\Python311\python.exe
Python version: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]

Young child boost parameters exist!
Default boost amount: $0


## Test 1: Household with Young Child (Age 3)

Setup:
- Married couple
- One child age 3 (eligible for young child boost)
- Income: $50,000
- CTC amount: $1,000
- Young child boost: $500
- Young child boost age limit: 6

Expected: Should receive $1,500 total CTC ($1,000 base + $500 young child boost)

In [2]:
# Build household with a 3-year-old child
household_young_child = build_household_situation(
    age_head=35,
    age_spouse=32,
    dependent_ages=[3],
    year=2026,
    with_axes=False
)

# Set AGI
household_young_child["tax_units"]["your tax unit"]["adjusted_gross_income"] = {"2026": 50000}

# Create reform with young child boost
reform_with_boost = create_custom_reform(
    ctc_amount=1000,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,  # Make fully refundable
    ctc_young_child_boost_amount=500,
    ctc_young_child_boost_age_limit=6,
    enable_exemption_reform=False
)

# Run baseline and reform simulations
sim_baseline = Simulation(situation=household_young_child)
sim_reform = Simulation(situation=household_young_child, reform=reform_with_boost)

# Calculate net incomes
net_income_baseline = sim_baseline.calculate("household_net_income", map_to="household", period=2026)[0]
net_income_reform = sim_reform.calculate("household_net_income", map_to="household", period=2026)[0]

benefit = net_income_reform - net_income_baseline

print(f"Baseline net income: ${net_income_baseline:,.2f}")
print(f"Reform net income: ${net_income_reform:,.2f}")
print(f"Total benefit: ${benefit:,.2f}")
print(f"\nExpected benefit: $1,500 (base $1,000 + boost $500)")
print(f"Match: {abs(benefit - 1500) < 10}")

Baseline net income: $28,105.77
Reform net income: $29,605.77
Total benefit: $1,500.00

Expected benefit: $1,500 (base $1,000 + boost $500)
Match: True


## Test 2: Household with Older Child (Age 10)

Setup:
- Same as Test 1, but child is age 10 (NOT eligible for young child boost)

Expected: Should receive $1,000 total CTC (base only, no boost)

In [3]:
# Build household with a 10-year-old child
household_older_child = build_household_situation(
    age_head=35,
    age_spouse=32,
    dependent_ages=[10],
    year=2026,
    with_axes=False
)

# Set AGI
household_older_child["tax_units"]["your tax unit"]["adjusted_gross_income"] = {"2026": 50000}

# Run simulations with same reform
sim_baseline_older = Simulation(situation=household_older_child)
sim_reform_older = Simulation(situation=household_older_child, reform=reform_with_boost)

# Calculate net incomes
net_income_baseline_older = sim_baseline_older.calculate("household_net_income", map_to="household", period=2026)[0]
net_income_reform_older = sim_reform_older.calculate("household_net_income", map_to="household", period=2026)[0]

benefit_older = net_income_reform_older - net_income_baseline_older

print(f"Baseline net income: ${net_income_baseline_older:,.2f}")
print(f"Reform net income: ${net_income_reform_older:,.2f}")
print(f"Total benefit: ${benefit_older:,.2f}")
print(f"\nExpected benefit: $1,000 (base only, no boost)")
print(f"Match: {abs(benefit_older - 1000) < 10}")

Baseline net income: $10,106.88
Reform net income: $11,106.88
Total benefit: $1,000.00

Expected benefit: $1,000 (base only, no boost)
Match: True


## Test 3: Household with Multiple Children of Different Ages

Setup:
- Married couple
- Three children: ages 2, 5, and 12
- Income: $75,000
- CTC amount: $1,000
- Young child boost: $500
- Young child boost age limit: 6

Expected: 
- Child age 2: $1,500 (base + boost)
- Child age 5: $1,500 (base + boost)
- Child age 12: $1,000 (base only)
- Total: $4,000

In [4]:
# Build household with multiple children
household_multiple = build_household_situation(
    age_head=40,
    age_spouse=38,
    dependent_ages=[2, 5, 12],
    year=2026,
    with_axes=False
)

# Set AGI
household_multiple["tax_units"]["your tax unit"]["adjusted_gross_income"] = {"2026": 75000}

# Run simulations
sim_baseline_multiple = Simulation(situation=household_multiple)
sim_reform_multiple = Simulation(situation=household_multiple, reform=reform_with_boost)

# Calculate net incomes
net_income_baseline_multiple = sim_baseline_multiple.calculate("household_net_income", map_to="household", period=2026)[0]
net_income_reform_multiple = sim_reform_multiple.calculate("household_net_income", map_to="household", period=2026)[0]

benefit_multiple = net_income_reform_multiple - net_income_baseline_multiple

print(f"Baseline net income: ${net_income_baseline_multiple:,.2f}")
print(f"Reform net income: ${net_income_reform_multiple:,.2f}")
print(f"Total benefit: ${benefit_multiple:,.2f}")
print(f"\nExpected benefit: $4,000")
print(f"  - Child age 2: $1,500 (base + boost)")
print(f"  - Child age 5: $1,500 (base + boost)")
print(f"  - Child age 12: $1,000 (base only)")
print(f"Match: {abs(benefit_multiple - 4000) < 10}")

Baseline net income: $34,713.50
Reform net income: $38,713.50
Total benefit: $4,000.00

Expected benefit: $4,000
  - Child age 2: $1,500 (base + boost)
  - Child age 5: $1,500 (base + boost)
  - Child age 12: $1,000 (base only)
Match: True


## Test 4: Verify Young Child Boost WITHOUT Base CTC

This tests that the boost is properly isolated and can be calculated separately.

In [5]:
# Create reform with ONLY young child boost (no base CTC)
reform_boost_only = create_custom_reform(
    ctc_amount=0,  # No base CTC
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=500,
    ctc_young_child_boost_age_limit=6,
    enable_exemption_reform=False
)

# Test with young child household
sim_boost_only = Simulation(situation=household_young_child, reform=reform_boost_only)
net_income_boost_only = sim_boost_only.calculate("household_net_income", map_to="household", period=2026)[0]

benefit_boost_only = net_income_boost_only - net_income_baseline

print(f"Baseline net income: ${net_income_baseline:,.2f}")
print(f"Reform net income (boost only): ${net_income_boost_only:,.2f}")
print(f"Benefit from boost only: ${benefit_boost_only:,.2f}")
print(f"\nExpected benefit: $500 (boost only, no base)")
print(f"Match: {abs(benefit_boost_only - 500) < 10}")

Baseline net income: $28,105.77
Reform net income (boost only): $28,605.77
Benefit from boost only: $500.00

Expected benefit: $500 (boost only, no base)
Match: True


## Test 5: Statewide Impact with Young Child Boost

Test the aggregate impact calculation to ensure young child boost affects statewide metrics.

In [6]:
print("Testing statewide impact WITHOUT young child boost...")
reform_no_boost = create_custom_reform(
    ctc_amount=1000,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=0,  # NO boost
    ctc_young_child_boost_age_limit=6,
    enable_exemption_reform=False
)

impact_no_boost = calculate_aggregate_impact(reform_no_boost)

print(f"\nTotal cost (no boost): ${impact_no_boost['total_cost']:,.0f}")
print(f"Beneficiary count (no boost): {impact_no_boost['beneficiaries']:,.0f}")
print(f"Average benefit (no boost): ${impact_no_boost['avg_benefit']:,.2f}")

Testing statewide impact WITHOUT young child boost...

Total cost (no boost): $205,316,095
Beneficiary count (no boost): 122,360
Average benefit (no boost): $1,677.97


In [7]:
print("Testing statewide impact WITH young child boost...")
reform_with_boost_aggregate = create_custom_reform(
    ctc_amount=1000,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=500,  # WITH boost
    ctc_young_child_boost_age_limit=6,
    enable_exemption_reform=False
)

impact_with_boost = calculate_aggregate_impact(reform_with_boost_aggregate)

print(f"\nTotal cost (with boost): ${impact_with_boost['total_cost']:,.0f}")
print(f"Beneficiary count (with boost): {impact_with_boost['beneficiaries']:,.0f}")
print(f"Average benefit (with boost): ${impact_with_boost['avg_benefit']:,.2f}")

print(f"\n=== COMPARISON ===")
cost_increase = impact_with_boost['total_cost'] - impact_no_boost['total_cost']
avg_benefit_increase = impact_with_boost['avg_benefit'] - impact_no_boost['avg_benefit']

print(f"Cost increase from young child boost: ${cost_increase:,.0f}")
print(f"Average benefit increase: ${avg_benefit_increase:,.2f}")
print(f"\nYoung child boost is working: {cost_increase > 0}")

Testing statewide impact WITH young child boost...

Total cost (with boost): $236,963,970
Beneficiary count (with boost): 122,360
Average benefit (with boost): $1,936.61

=== COMPARISON ===
Cost increase from young child boost: $31,647,876
Average benefit increase: $258.65

Young child boost is working: True


## Test 6: Check PolicyEngine Variables Directly

Let's look at the actual CTC variables being calculated by PolicyEngine.

In [8]:
# Check what variables are available related to RI CTC
print("Testing individual CTC components...\n")

# Use the young child household
sim_reform_check = Simulation(situation=household_young_child, reform=reform_with_boost)

# Try to get RI CTC components
try:
    ri_ctc = sim_reform_check.calculate("ri_ctc", period=2026)
    print(f"RI CTC amount: ${ri_ctc[0]:,.2f}")
except:
    print("Could not calculate 'ri_ctc' variable")

try:
    state_tax_before_credits = sim_reform_check.calculate("ri_income_tax_before_credits", period=2026)
    print(f"RI tax before credits: ${state_tax_before_credits[0]:,.2f}")
except:
    print("Could not calculate 'ri_income_tax_before_credits'")

try:
    state_tax_after = sim_reform_check.calculate("ri_income_tax", period=2026)
    print(f"RI tax after credits: ${state_tax_after[0]:,.2f}")
except:
    print("Could not calculate 'ri_income_tax'")

try:
    household_refundable_tax_credits = sim_reform_check.calculate("household_refundable_tax_credits", period=2026)
    print(f"Household refundable credits: ${household_refundable_tax_credits[0]:,.2f}")
except:
    print("Could not calculate household_refundable_tax_credits")

Testing individual CTC components...

Could not calculate 'ri_ctc' variable
Could not calculate 'ri_income_tax_before_credits'
RI tax after credits: $-1,003.12
Household refundable credits: $1,500.00


## Test 7: Statewide Impact with Specific Parameters

Testing with:
- CTC amount: $312
- Young child boost: $1,000
- Exemption reform enabled but amount set to $0
- Compare impact WITHOUT boost vs WITH boost

In [10]:
print("Test 7: Statewide impact with CTC=$312, exemption=$0")
print("=" * 70)

print("\n1. Testing WITHOUT young child boost...")
reform_test7_no_boost = create_custom_reform(
    ctc_amount=312,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=0,  # NO boost
    ctc_young_child_boost_age_limit=6,
    enable_exemption_reform=True,
    exemption_amount=0,
    exemption_age_limit_enabled=True,
    exemption_age_threshold=18,
)

impact_test7_no_boost = calculate_aggregate_impact(reform_test7_no_boost)

print(f"\nTotal cost (no boost): ${impact_test7_no_boost['total_cost']:,.0f}")
print(f"Beneficiary count (no boost): {impact_test7_no_boost['beneficiaries']:,.0f}")
print(f"Average benefit (no boost): ${impact_test7_no_boost['avg_benefit']:,.2f}")

print("\n2. Testing WITH young child boost ($1,000)...")
reform_test7_with_boost = create_custom_reform(
    ctc_amount=312,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=1000,  # WITH boost
    ctc_young_child_boost_age_limit=6,
    enable_exemption_reform=True,
    exemption_amount=0,
    exemption_age_limit_enabled=True,
    exemption_age_threshold=18,
)

impact_test7_with_boost = calculate_aggregate_impact(reform_test7_with_boost)

print(f"\nTotal cost (with boost): ${impact_test7_with_boost['total_cost']:,.0f}")
print(f"Beneficiary count (with boost): ${impact_test7_with_boost['beneficiaries']:,.0f}")
print(f"Average benefit (with boost): ${impact_test7_with_boost['avg_benefit']:,.2f}")

print(f"\n{'=' * 70}")
print("COMPARISON: Impact of Young Child Boost")
print('=' * 70)

cost_increase_test7 = impact_test7_with_boost['total_cost'] - impact_test7_no_boost['total_cost']
avg_benefit_increase_test7 = impact_test7_with_boost['avg_benefit'] - impact_test7_no_boost['avg_benefit']

print(f"\nCost increase from young child boost: ${cost_increase_test7:,.0f}")
print(f"Average benefit increase: ${avg_benefit_increase_test7:,.2f}")
print(f"Percent increase in cost: {(cost_increase_test7 / impact_test7_no_boost['total_cost'] * 100):.1f}%")
print(f"\nYoung child boost is working: {cost_increase_test7 > 0}")

Test 7: Statewide impact with CTC=$312, exemption=$0

1. Testing WITHOUT young child boost...

Total cost (no boost): $36,832,095
Beneficiary count (no boost): 122,360
Average benefit (no boost): $304.29

2. Testing WITH young child boost ($1,000)...

Total cost (with boost): $100,127,847
Beneficiary count (with boost): $122,360
Average benefit (with boost): $825.63

COMPARISON: Impact of Young Child Boost

Cost increase from young child boost: $63,295,751
Average benefit increase: $521.34
Percent increase in cost: 171.8%

Young child boost is working: True


In [5]:
print("Test 9: Comprehensive Impact Analysis")
print("=" * 80)

reform_test9 = create_custom_reform(
    ctc_amount=300,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=300,
    ctc_young_child_boost_age_limit=6,
    ctc_phaseout_rate=0.012,  # 0.012%
    ctc_phaseout_thresholds={
        "SINGLE": 250000,
        "JOINT": 250000,
        "HEAD_OF_HOUSEHOLD": 250000,
        "SURVIVING_SPOUSE": 250000,
        "SEPARATE": 250000,
    },
    enable_exemption_reform=True,
    exemption_amount=0,
    exemption_age_limit_enabled=True,
    exemption_age_threshold=18,
)

print("\nCalculating comprehensive impact metrics...")
print("This may take a few moments...")
impact_test9 = calculate_aggregate_impact(reform_test9)

print("\n" + "=" * 80)
print("STATEWIDE FISCAL IMPACT")
print("=" * 80)

print(f"\nTotal Annual Cost: ${impact_test9['total_cost']:,.0f}")
print(f"Households Benefiting: {impact_test9['beneficiaries']:,.0f}")
print(f"Average Benefit per Household: ${impact_test9['avg_benefit']:,.2f}")
print(f"Children Affected: {impact_test9['children_affected']:,.0f}")

print("\n" + "=" * 80)
print("WINNERS & LOSERS")
print("=" * 80)

print(f"\nWinners (better off): {impact_test9['winners']:,.0f} households ({impact_test9['winners_rate']:.2f}%)")
print(f"Losers (worse off): {impact_test9['losers']:,.0f} households ({impact_test9['losers_rate']:.2f}%)")

print("\n" + "=" * 80)
print("POVERTY IMPACT")
print("=" * 80)

print(f"\nOverall Poverty Rate:")
print(f"  Baseline: {impact_test9['poverty_baseline_rate']:.2f}%")
print(f"  With Reform: {impact_test9['poverty_reform_rate']:.2f}%")
print(f"  Change: {impact_test9['poverty_rate_change']:+.2f} percentage points")
print(f"  Percent Change: {impact_test9['poverty_percent_change']:+.1f}%")

print(f"\nChild Poverty Rate:")
print(f"  Baseline: {impact_test9['child_poverty_baseline_rate']:.2f}%")
print(f"  With Reform: {impact_test9['child_poverty_reform_rate']:.2f}%")
print(f"  Change: {impact_test9['child_poverty_rate_change']:+.2f} percentage points")
print(f"  Percent Change: {impact_test9['child_poverty_percent_change']:+.1f}%")

print("\n" + "=" * 80)
print("IMPACT BY INCOME BRACKET")
print("=" * 80)

for bracket in impact_test9['by_income_bracket']:
    print(f"\n{bracket['bracket']}:")
    print(f"  Households Affected: {bracket['beneficiaries']:,.0f}")
    print(f"  Total Cost: ${bracket['total_cost']:,.0f}")
    print(f"  Average Benefit: ${bracket['avg_benefit']:,.2f}")

print("\n" + "=" * 80)
print("COMPONENT BREAKDOWN")
print("=" * 80)

print("\nPolicy Parameters:")
print(f"  CTC Base Amount: $300")
print(f"  Young Child Boost: $300")
print(f"  Phase-out Rate: 0.012% (starts at $250k)")
print(f"  Dependent Exemption: $0")

print("\nExpected benefit for household with young child (under $250k income):")
print(f"  Base CTC: $300")
print(f"  Young Child Boost: $300")
print(f"  Subtotal: $600")
print(f"  Exemption impact: ~-$195")
print(f"  Expected Net: ~$405")

Test 9: Comprehensive Impact Analysis

Calculating comprehensive impact metrics...
This may take a few moments...

STATEWIDE FISCAL IMPACT

Total Annual Cost: $51,199,974
Households Benefiting: 117,829
Average Benefit per Household: $428.01
Children Affected: 198,431

WINNERS & LOSERS

Winners (better off): 39,353,069 households (37.95%)
Losers (worse off): 652,091 households (0.63%)

POVERTY IMPACT

Overall Poverty Rate:
  Baseline: 16.97%
  With Reform: 16.76%
  Change: -0.22 percentage points
  Percent Change: -1.3%

Child Poverty Rate:
  Baseline: 18.78%
  With Reform: 18.19%
  Change: -0.59 percentage points
  Percent Change: -3.1%

IMPACT BY INCOME BRACKET

Under $50k:
  Households Affected: 26,148
  Total Cost: $15,013,877
  Average Benefit: $574.19

$50k-$100k:
  Households Affected: 30,875
  Total Cost: $12,573,208
  Average Benefit: $407.23

$100k-$150k:
  Households Affected: 20,989
  Total Cost: $7,959,224
  Average Benefit: $379.20

$150k-$200k:
  Households Affected: 21,6

## Test 9: Comprehensive Impact Analysis

Testing with:
- CTC base amount: $300
- Young child boost: $300
- Exemption: $0
- Phase-out rate: 0.012% (0.00012)
- Phase-out threshold: $250,000 for ALL filing statuses

This test shows all metrics from the app's impact analysis section.

In [3]:
print("Test 8: Statewide impact with phase-out thresholds")
print("=" * 70)

print("\n1. Testing WITHOUT young child boost (with phase-out)...")
reform_test8_no_boost = create_custom_reform(
    ctc_amount=312,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=0,  # NO boost
    ctc_young_child_boost_age_limit=6,
    ctc_phaseout_rate=0.05,  # 5% phase-out
    ctc_phaseout_thresholds={
        "SINGLE": 290000,
        "JOINT": 580000,
        "HEAD_OF_HOUSEHOLD": 290000,
        "SURVIVING_SPOUSE": 580000,
        "SEPARATE": 290000,
    },
    enable_exemption_reform=True,
    exemption_amount=0,
    exemption_age_limit_enabled=True,
    exemption_age_threshold=18,
)

impact_test8_no_boost = calculate_aggregate_impact(reform_test8_no_boost)

print(f"\nTotal cost (no boost): ${impact_test8_no_boost['total_cost']:,.0f}")
print(f"Beneficiary count (no boost): {impact_test8_no_boost['beneficiaries']:,.0f}")
print(f"Average benefit (no boost): ${impact_test8_no_boost['avg_benefit']:,.2f}")

print("\n2. Testing WITH young child boost ($1,000, with phase-out)...")
reform_test8_with_boost = create_custom_reform(
    ctc_amount=312,
    ctc_age_limit=18,
    ctc_refundability_cap=999999,
    ctc_young_child_boost_amount=1000,  # WITH boost
    ctc_young_child_boost_age_limit=6,
    ctc_phaseout_rate=0.05,  # 5% phase-out
    ctc_phaseout_thresholds={
        "SINGLE": 290000,
        "JOINT": 580000,
        "HEAD_OF_HOUSEHOLD": 290000,
        "SURVIVING_SPOUSE": 580000,
        "SEPARATE": 290000,
    },
    enable_exemption_reform=True,
    exemption_amount=0,
    exemption_age_limit_enabled=True,
    exemption_age_threshold=18,
)

impact_test8_with_boost = calculate_aggregate_impact(reform_test8_with_boost)

print(f"\nTotal cost (with boost): ${impact_test8_with_boost['total_cost']:,.0f}")
print(f"Beneficiary count (with boost): {impact_test8_with_boost['beneficiaries']:,.0f}")
print(f"Average benefit (with boost): ${impact_test8_with_boost['avg_benefit']:,.2f}")

print(f"\n{'=' * 70}")
print("COMPARISON: Impact of Young Child Boost (with phase-out)")
print('=' * 70)

cost_increase_test8 = impact_test8_with_boost['total_cost'] - impact_test8_no_boost['total_cost']
avg_benefit_increase_test8 = impact_test8_with_boost['avg_benefit'] - impact_test8_no_boost['avg_benefit']

print(f"\nCost increase from young child boost: ${cost_increase_test8:,.0f}")
print(f"Average benefit increase: ${avg_benefit_increase_test8:,.2f}")
print(f"Percent increase in cost: {(cost_increase_test8 / impact_test8_no_boost['total_cost'] * 100):.1f}%")
print(f"\nYoung child boost is working: {cost_increase_test8 > 0}")

print(f"\n{'=' * 70}")
print("COMPARISON: Phase-out Impact")
print('=' * 70)

# Compare test 7 (no phase-out) vs test 8 (with phase-out)
cost_reduction_phaseout = impact_test7_with_boost['total_cost'] - impact_test8_with_boost['total_cost']
print(f"\nCost without phase-out (Test 7): ${impact_test7_with_boost['total_cost']:,.0f}")
print(f"Cost with phase-out (Test 8): ${impact_test8_with_boost['total_cost']:,.0f}")
print(f"Cost reduction from phase-out: ${cost_reduction_phaseout:,.0f}")
print(f"Percent reduction: {(cost_reduction_phaseout / impact_test7_with_boost['total_cost'] * 100):.1f}%")

Test 8: Statewide impact with phase-out thresholds

1. Testing WITHOUT young child boost (with phase-out)...


KeyboardInterrupt: 

## Test 8: Statewide Impact with Phase-out Thresholds

Testing with:
- CTC amount: $312
- Young child boost: $1,000
- Exemption: $0
- **Phase-out rate: 5% (0.05)**
- **Phase-out thresholds:**
  - Single, HOH, Separate: $290,000
  - Joint, Surviving Spouse: $580,000